### 데이터 수집

In [ ]:
!uv add torch torch_geometric pandas networkx numpy python-dotenv


In [4]:
import requests
import time
import pandas as pd

from datetime import datetime, timezone
from dotenv import load_dotenv
import os



# .env 파일 로드
load_dotenv()

# --------------------------------
# 설정
# --------------------------------
API_KEY = os.getenv('ETHEREUM_API_KEY')  # Etherscan API 키 입력
CONTRACT = os.getenv('ETHEREUM_TOKEN_CONTRACT_ADDRESS')  # PicaArtMoney 토큰 컨트랙트
START_DATE = "2020-10-13"
END_DATE   = "2024-08-07"

# --------------------------
# 유틸: UTC 타임스탬프 변환
# --------------------------
def to_ts_utc(dstr: str) -> int:
    y, m, d = map(int, dstr.split("-"))
    return int(datetime(y, m, d, tzinfo=timezone.utc).timestamp())

start_ts = to_ts_utc(START_DATE)
end_ts   = to_ts_utc(END_DATE)

# --------------------------
# 블록 번호 by timestamp
# --------------------------
def get_block_number_by_timestamp(ts: int, closest="before") -> int:
    url = "https://api.etherscan.io/api"
    params = {
        "module": "block",
        "action": "getblocknobytime",
        "timestamp": ts,
        "closest": closest,
        "apikey": API_KEY
    }
    r = requests.get(url, params=params, timeout=30).json()
    return int(r["result"])

start_block = get_block_number_by_timestamp(start_ts, "after")
end_block   = get_block_number_by_timestamp(end_ts, "before")
print(f"[INFO] Block range: {start_block} ~ {end_block}")

# --------------------------
# 페이지 호출 (재시도 포함)
# --------------------------
def fetch_transfers_page(page, offset, start_block, end_block):
    url = "https://api.etherscan.io/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": CONTRACT,
        "startblock": start_block,
        "endblock": end_block,
        "page": page,
        "offset": offset,
        "sort": "asc",
        "apikey": API_KEY
    }
    # 간단 재시도
    for i in range(5):
        try:
            resp = requests.get(url, params=params, timeout=30).json()
            # status=="1"이면 결과, "0"이면 없음(또는 rate limit)
            if resp.get("status") == "1":
                return resp["result"]
            # "Max rate limit reached" 같은 메시지면 잠깐 대기 후 재시도
            if "Max rate limit reached" in resp.get("message",""):
                time.sleep(1.0 * (i+1))
                continue
            return []
        except Exception:
            time.sleep(1.0 * (i+1))
    return []

# --------------------------
# 범위 수집 (빈 청크도 로그)
# --------------------------
all_txs = []
step = 200_000        # 블록 청크 크기
offset = 10_000       # 페이지 크기
sleep_s = 0.25        # rate limit 보호
current = start_block

# (선택) 처리한 청크 목록 보관해 누락 시각화
processed_ranges = []

while current <= end_block:
    to_block = min(current + step - 1, end_block)
    processed_ranges.append((current, to_block))

    page = 1
    new_cnt = 0
    while True:
        txs = fetch_transfers_page(page, offset, current, to_block)
        if not txs:
            # 첫 페이지부터 빈 경우 → 이 청크엔 트랜잭션 없음을 명시
            if page == 1:
                print(f"[INFO] Blocks {current}–{to_block}, Page {page}, NO TX")
            break

        # 기간 필터
        for tx in txs:
            ts = int(tx.get("timeStamp", 0))
            if start_ts <= ts <= end_ts:
                all_txs.append(tx)
                new_cnt += 1

        print(f"[INFO] Blocks {current}–{to_block}, Page {page}, Added {new_cnt}, Total {len(all_txs)}")
        page += 1
        time.sleep(sleep_s)

        # 마지막 페이지(=반환건수 < offset) 판단 → 더 이상 이 청크에서 페이지 없음
        if len(txs) < offset:
            break

    current = to_block + 1  # 다음 청크로

# --------------------------
# DataFrame & 중복 제거
# --------------------------
df = pd.DataFrame(all_txs)

# 어떤 컬럼이 있는지 확인
print("[INFO] Columns:", df.columns.tolist())

# 존재하는 컬럼만으로 중복 제거
subset_cols = [c for c in ["hash", "logIndex", "transactionIndex"] if c in df.columns]
if subset_cols:
    df.drop_duplicates(subset=subset_cols, inplace=True)
else:
    df.drop_duplicates(subset=["hash"], inplace=True)

# (선택) amount 컬럼 추가
if {"value","tokenDecimal"} <= set(df.columns):
    df["amount"] = (df["value"].astype("int64") /
                    (10 ** df["tokenDecimal"].astype(int)))
else:
    df["amount"] = 0.0

out_path = "picaartmoney_transactions_full.csv"
df.to_csv(out_path, index=False)
print(f"[INFO] Finished. Total collected: {len(df)} transactions → {out_path}")

# --------------------------
# (선택) 누락 구간 시각 확인용
# --------------------------
# 빈 청크도 NO TX 로그가 찍히므로, 처리 범위가 연속적이라면 누락은 없습니다.
# 그래도 안심용으로 연속성 체크:
gaps = []
for (a1,b1),(a2,b2) in zip(processed_ranges, processed_ranges[1:]):
    if a2 != b1 + 1:
        gaps.append((b1+1, a2-1))
if gaps:
    print("[WARN] Detected block gaps in iteration (should not happen):", gaps)
else:
    print("[INFO] No block range gaps in iteration (every chunk covered).")

[INFO] Block range: 11043877 ~ 20472970
[INFO] Blocks 11043877–11243876, Page 1, Added 3828, Total 3828
[INFO] Blocks 11243877–11443876, Page 1, Added 238, Total 4066
[INFO] Blocks 11443877–11643876, Page 1, Added 321, Total 4387
[INFO] Blocks 11643877–11843876, Page 1, Added 2973, Total 7360
[INFO] Blocks 11843877–12043876, Page 1, Added 2121, Total 9481
[INFO] Blocks 12043877–12243876, Page 1, Added 5185, Total 14666
[INFO] Blocks 12243877–12443876, Page 1, Added 2273, Total 16939
[INFO] Blocks 12443877–12643876, Page 1, Added 1210, Total 18149
[INFO] Blocks 12643877–12843876, Page 1, Added 3398, Total 21547
[INFO] Blocks 12843877–13043876, Page 1, Added 1086, Total 22633
[INFO] Blocks 13043877–13243876, Page 1, Added 220, Total 22853
[INFO] Blocks 13243877–13443876, Page 1, Added 144, Total 22997
[INFO] Blocks 13443877–13643876, Page 1, Added 3338, Total 26335
[INFO] Blocks 13643877–13843876, Page 1, Added 38, Total 26373
[INFO] Blocks 13843877–14043876, Page 1, Added 10, Total 2638

### 수집 데이터 전처리

In [5]:
# ============================================================
# 0) Imports
# ============================================================
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

from torch_geometric.data import Data
from torch_geometric.nn import GATConv, GAE
from torch_geometric.utils import to_undirected, negative_sampling
from torch_geometric.transforms import RandomLinkSplit

import networkx as nx
from decimal import Decimal, getcontext
from datetime import datetime, timezone
from collections import defaultdict, Counter


In [6]:

# ------------------------------------------------------------
# 파일 경로
CSV_PATH = "picaartmoney_transactions_full.csv"
USE_COLS = [
    'blockNumber','timeStamp','hash','nonce','blockHash','from','contractAddress','to','value',
    'tokenName','tokenSymbol','tokenDecimal','transactionIndex','gas','gasPrice','gasUsed',
    'cumulativeGasUsed','input','confirmations'
]

# 소수 정밀도 (ETH/토큰 소수 처리 안전하게)
getcontext().prec = 50

In [7]:
# ------------------------------------------------------------
# 0) CSV 로드 & 전처리
df = pd.read_csv(CSV_PATH, usecols=USE_COLS, dtype=str)

# 결측/공백 안전 처리
for col in ['from', 'to']:
    df[col] = df[col].fillna('').str.strip().str.lower()

# timestamp -> int / datetime
df['timeStamp'] = pd.to_numeric(df['timeStamp'], errors='coerce')
df = df.dropna(subset=['timeStamp'])
df['timestamp_dt'] = pd.to_datetime(df['timeStamp'], unit='s', utc=True)

# 숫자형 컬럼 변환
for c in ['value', 'tokenDecimal', 'gas', 'gasPrice', 'gasUsed']:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

# value → 정규화 (value / 10**tokenDecimal)
def safe_value(row):
    try:
        dec = int(row['tokenDecimal'])
        val = Decimal(int(row['value'])) / (Decimal(10) ** dec)
        return val
    except Exception:
        return Decimal(0)

df['value_float'] = df.apply(safe_value, axis=1)

# gasUsed 우선 사용 (없으면 gas)
df['gas_used'] = np.where(df['gasUsed'] > 0, df['gasUsed'], df['gas'])

# 송/수신이 비어있거나 동일지갑 자기전송은 제외(원하면 포함 가능)
df = df[(df['from'] != '') & (df['to'] != '') & (df['from'] != df['to'])]

df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,...,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations,timestamp_dt,value_float,gas_used
0,11085582,1603098539,0x328008e17407b6ba014295bfe5069ba4f60f1296aea0...,0,0x7185066660404b22f7f1cac0862c8a7f5c1ef99e4a1f...,0x0000000000000000000000000000000000000000,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xd28493e737fbcc957f3716143ed6e40f40357b51,1000000000,PicaArtMoney,...,251,1603895,36000000000,1603895,11088223,deprecated,12287913,2020-10-19 09:08:59+00:00,1000000000,1603895
1,11091306,1603174037,0xa8352e8094fb444e9bfa9a4a6d8011502a0f4655ad38...,1,0x6902d2e0773fd0bad116b753b98e37d61d7a4b532d71...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xfa9b57cbe5b7bd63b436dcf205c15222b510ff27,150000000,PicaArtMoney,...,185,53110,26000000000,53110,12360057,deprecated,12282189,2020-10-20 06:07:17+00:00,150000000,53110
2,11091306,1603174037,0x36195c14399493bdf43387ded77d87d3b5e98c94a138...,2,0x6902d2e0773fd0bad116b753b98e37d61d7a4b532d71...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0x32f042b0b01f10247493a950456f4c4304d46ba5,150000000,PicaArtMoney,...,186,53110,26000000000,53110,12413167,deprecated,12282189,2020-10-20 06:07:17+00:00,150000000,53110
3,11091306,1603174037,0xbf52a2c6de897287b5c6740c891a71a5122c69566bdb...,3,0x6902d2e0773fd0bad116b753b98e37d61d7a4b532d71...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xd4b394c60bb55f80df30dac87b6f92be34739332,300000000,PicaArtMoney,...,187,53098,26000000000,53098,12466265,deprecated,12282189,2020-10-20 06:07:17+00:00,300000000,53098
4,11091313,1603174087,0x246a4998919d4c369ff0735ea372729a9e06199055e1...,4,0xe18ff0b2e114d21a3f5cf8c2976342cea2fc558254ee...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xc32b1345acae345c595d3bbcf62e14e5f3020456,200000000,PicaArtMoney,...,100,53098,27000000000,53098,5923677,deprecated,12282182,2020-10-20 06:08:07+00:00,200000000,53098


### 그래프데이터 생성

In [8]:
# 1) 기본엣지 그래프 (MultiDiGraph) 구축
G_base = nx.MultiDiGraph()
# 노드 추가(지갑주소)
nodes = pd.unique(pd.concat([df['from'], df['to']], ignore_index=True))
G_base.add_nodes_from(nodes)

# 엣지 추가(트랜잭션 단위)
for _, r in df.iterrows():
    G_base.add_edge(
        r['from'], r['to'],
        key=r['hash'],
        hash=r['hash'],
        timestamp=int(r['timeStamp']),
        timestamp_dt=r['timestamp_dt'],
        value_float=r['value_float'],
        gas_used=int(r['gas_used'])
    )

In [ ]:
# 2) 총괄엣지 그래프 (무방향, 계좌쌍 집계)
# net_value: sum(A->B) - sum(B->A), tx_count: 총 거래 횟수, first_tx_time: 최소 시각
pair_sum_ab = defaultdict(Decimal)   # sum A->B
pair_sum_ba = defaultdict(Decimal)   # sum B->A
pair_count = Counter()
pair_first_ts = dict()

for _, r in df.iterrows():
    a, b = r['from'], r['to']
    key = tuple(sorted((a, b)))
    pair_count[key] += 1
    ts = int(r['timeStamp'])
    if key not in pair_first_ts or ts < pair_first_ts[key]:
        pair_first_ts[key] = ts
    # 방향 합계
    if a < b:
        # 저장은 작은주소,큰주소 기준으로 해두고 방향은 따로 기록
        pair_sum_ab[key] += r['value_float']  # a(작은?)→b(큰?)가 아닐 수 있어 아래에서 다시 방향판단
    else:
        pair_sum_ba[key] += r['value_float']

# 더 명확하게 재집계:
pair_dir_sum = defaultdict(lambda: {'ab': Decimal(0), 'ba': Decimal(0)})
for _, r in df.iterrows():
    a, b = r['from'], r['to']
    key = tuple(sorted((a, b)))
    if (a, b) == key:
        pair_dir_sum[key]['ab'] += r['value_float']
    else:
        pair_dir_sum[key]['ba'] += r['value_float']

H_summary = nx.Graph()
H_summary.add_nodes_from(nodes)

for key in pair_count.keys():
    a, b = key
    ab = pair_dir_sum[key]['ab']
    ba = pair_dir_sum[key]['ba']
    net_value = ab - ba
    first_ts = pair_first_ts[key]
    H_summary.add_edge(
        a, b,
        net_value=float(net_value),         # CSV 저장 편의 위해 float 캐스팅(정밀 보존 원하면 str(net_value))
        tx_count=int(pair_count[key]),
        first_tx_time=int(first_ts)
    )

In [ ]:

# 3) 노드 특성 계산 (29개)
# 준비: 방향 기준별 집계
# 노드별 in/out 트랜잭션 수/금액/가스
in_count = Counter()
out_count = Counter()
in_amount_sum = defaultdict(Decimal)
out_amount_sum = defaultdict(Decimal)
in_gas_sum = Counter()
out_gas_sum = Counter()

# 노드별 타임스탬프 모음
node_timestamps = defaultdict(list)

# 노드별 in/out 이웃(상대방) 집합
in_neighbors = defaultdict(set)
out_neighbors = defaultdict(set)

for _, r in df.iterrows():
    s, t = r['from'], r['to']
    val = r['value_float']
    gasu = int(r['gas_used'])
    ts = int(r['timeStamp'])

    # 카운트/합계
    out_count[s] += 1
    in_count[t] += 1
    out_amount_sum[s] += val
    in_amount_sum[t] += val
    out_gas_sum[s] += gasu
    in_gas_sum[t] += gasu

    # 이웃
    out_neighbors[s].add(t)
    in_neighbors[t].add(s)

    # 타임스탬프
    node_timestamps[s].append(ts)
    node_timestamps[t].append(ts)

# 3-1) 유일 송/수신 상대 관련
# - "송신 트랜잭션의 유일한 상대방이 해당 노드인 노드들의 수 / 총 수량"
#   즉, X의 out_neighbors(X)가 {v}인 모든 X를 v 기준으로 집계
unique_sender_targets = defaultdict(list)  # v -> [X,...] where X sends only to v
for x, outs in out_neighbors.items():
    if len(outs) == 1:
        v = next(iter(outs))
        unique_sender_targets[v].append(x)

# 유일송신 총 수량(해당 X->v로 보낸 모든 금액 합)
unique_send_total_amount_to_v = defaultdict(Decimal)
for v, senders in unique_sender_targets.items():
    # df 필터 비용 줄이려면 사전 집계를 쓰는 것이 좋지만, 데이터 건수가 2.6만이라도 충분히 처리 가능
    mask = (df['from'].isin(senders)) & (df['to'] == v)
    if mask.any():
        unique_send_total_amount_to_v[v] = df.loc[mask, 'value_float'].sum()
    else:
        unique_send_total_amount_to_v[v] = Decimal(0)

# - "수신 트랜잭션의 유일한 상대발이 해당 노드인 노드들의 수 / 총 수량"
#   즉, Y의 in_neighbors(Y)가 {v}인 모든 Y를 v 기준으로 집계 (Y는 v에게서만 받음)
unique_receiver_sources = defaultdict(list)  # v -> [Y,...] where Y receives only from v
for y, ins in in_neighbors.items():
    if len(ins) == 1:
        v = next(iter(ins))
        unique_receiver_sources[v].append(y)

unique_recv_total_amount_from_v = defaultdict(Decimal)
for v, receivers in unique_receiver_sources.items():
    mask = (df['to'].isin(receivers)) & (df['from'] == v)
    if mask.any():
        unique_recv_total_amount_from_v[v] = df.loc[mask, 'value_float'].sum()
    else:
        unique_recv_total_amount_from_v[v] = Decimal(0)

# 3-2) 순환 거래수: 같은 SCC 안의 상대와 주고받은 트랜잭션 수
# (방향 그래프의 강결합요소 기반)
sccs = list(nx.strongly_connected_components(G_base))
comp_id = {}
for i, comp in enumerate(sccs):
    for n in comp:
        comp_id[n] = i

cycle_tx_count = Counter()
for u, v, k, d in G_base.edges(keys=True, data=True):
    if comp_id.get(u) == comp_id.get(v) and len(sccs[comp_id[u]]) > 1:
        cycle_tx_count[u] += 1
        cycle_tx_count[v] += 1

# 3-3) 양방향 상대 수: u<->v 모두 존재하는 상대 수
bidirectional_count = Counter()
# 빠른 판별: 무방향으로 변환 후, 각 이웃 중 실제로 양방향 존재하는지 체크
UG = G_base.to_undirected()
for n in G_base.nodes():
    cnt = 0
    for nbr in UG.neighbors(n):
        has_out = G_base.has_edge(n, nbr)
        has_in = G_base.has_edge(nbr, n)
        if has_out and has_in:
            cnt += 1
    bidirectional_count[n] = cnt

# 3-4) 중심성들
# Degree/Closeness/Betweenness: 무방향 그래프 기준
deg_centrality = nx.degree_centrality(UG)
# closeness는 연결요소 문제로 normalized=True 기본, 무방향에서 계산
close_centrality = nx.closeness_centrality(UG)
# betweenness: 계산 비용 큼. 노드 많으면 k-샘플링 사용 고려. 여기서는 정확 계산 시도.
bet_centrality = nx.betweenness_centrality(UG, normalized=True)

# PageRank: 방향 + 가중치(value_float)
# weight가 float이어야 해서 미리 edge attr 준비 필요 → 이미 value_float 있음
# MultiDiGraph이므로 가중치 합산 필요 → DiGraph로 합쳐서 가중치 누적
DG_weighted = nx.DiGraph()
for u, v, d in G_base.edges(data=True):
    w = float(d.get('value_float', 0))
    if w <= 0:
        continue
    if DG_weighted.has_edge(u, v):
        DG_weighted[u][v]['weight'] += w
    else:
        DG_weighted.add_edge(u, v, weight=w)

if DG_weighted.number_of_edges() > 0:
    pagerank = nx.pagerank(DG_weighted, weight='weight')
else:
    pagerank = {n: 0.0 for n in G_base.nodes()}

# 3-5) 나머지 지표들 조립
rows = []
for n in G_base.nodes():
    in_neigh = in_neighbors.get(n, set())
    out_neigh = out_neighbors.get(n, set())

    total_in_cnt = in_count.get(n, 0)
    total_out_cnt = out_count.get(n, 0)
    total_in_amt = in_amount_sum.get(n, Decimal(0))
    total_out_amt = out_amount_sum.get(n, Decimal(0))
    total_in_gas = in_gas_sum.get(n, 0)
    total_out_gas = out_gas_sum.get(n, 0)

    # 평균들(0 나눗셈 방지)
    avg_in_amt = (total_in_amt / total_in_cnt) if total_in_cnt > 0 else Decimal(0)
    avg_out_amt = (total_out_amt / total_out_cnt) if total_out_cnt > 0 else Decimal(0)
    avg_in_gas = (Decimal(total_in_gas) / total_in_cnt) if total_in_cnt > 0 else Decimal(0)
    avg_out_gas = (Decimal(total_out_gas) / total_out_cnt) if total_out_cnt > 0 else Decimal(0)

    # 가스 효율
    recv_gas_eff = (total_in_amt / Decimal(total_in_gas)) if total_in_gas > 0 else Decimal(0)
    send_gas_eff = (total_out_amt / Decimal(total_out_gas)) if total_out_gas > 0 else Decimal(0)

    # 활동 일수/일평균
    ts_list = node_timestamps.get(n, [])
    unique_days = set()
    for ts in ts_list:
        d = datetime.fromtimestamp(ts, tz=timezone.utc).date()
        unique_days.add(d)
    active_days = len(unique_days)
    total_tx_cnt = total_in_cnt + total_out_cnt
    total_amt_abs = total_in_amt + total_out_amt  # 필요시 abs 합으로 바꾸고 싶으면 수정
    tx_per_day = (total_tx_cnt / active_days) if active_days > 0 else 0
    amt_per_day = (total_amt_abs / Decimal(active_days)) if active_days > 0 else Decimal(0)

    # 유일 송/수신 관련
    uniq_send_nodes = unique_sender_targets.get(n, [])
    uniq_recv_nodes = unique_receiver_sources.get(n, [])

    uniq_send_count = len(uniq_send_nodes)
    uniq_recv_count = len(uniq_recv_nodes)
    uniq_send_amount = unique_send_total_amount_to_v.get(n, Decimal(0))
    uniq_recv_amount = unique_recv_total_amount_from_v.get(n, Decimal(0))

    # 순환 거래수/양방향 상대수
    cyc_cnt = cycle_tx_count.get(n, 0)
    bidi_cnt = bidirectional_count.get(n, 0)

    # 중심성
    deg_c = deg_centrality.get(n, 0.0)
    clo_c = close_centrality.get(n, 0.0)
    bet_c = bet_centrality.get(n, 0.0)
    pr = pagerank.get(n, 0.0)

    # 첫/마지막 트랜잭션 시각
    if ts_list:
        first_ts = min(ts_list)
        last_ts = max(ts_list)
    else:
        first_ts = None
        last_ts = None

    rows.append({
        'address': n,

        # 1~4 유일 송/수신 관련
        'unique_sender_nodes_count': uniq_send_count,                          # (1)
        'unique_receiver_nodes_count': uniq_recv_count,                        # (2)
        'unique_sender_total_amount': float(uniq_send_amount),                 # (3)
        'unique_receiver_total_amount': float(uniq_recv_amount),               # (4)

        # 5~6 순환/양방향
        'cycle_tx_count': int(cyc_cnt),                                        # (5)
        'bidirectional_counterparties': int(bidi_cnt),                         # (6)

        # 7~10 중심성
        'closeness_centrality': float(clo_c),                                  # (7)
        'betweenness_centrality': float(bet_c),                                # (8)
        'pagerank': float(pr),                                                 # (9)
        'degree_centrality': float(deg_c),                                     # (10)

        # 11~12 이웃수(방향)
        'in_neighbor_count': len(in_neigh),                                    # (11)
        'out_neighbor_count': len(out_neigh),                                  # (12)

        # 13~22 in/out 트랜잭션 집계
        'total_in_tx_count': int(total_in_cnt),                                # (13)
        'total_out_tx_count': int(total_out_cnt),                              # (14)
        'total_in_amount': float(total_in_amt),                                 # (15)
        'total_out_amount': float(total_out_amt),                               # (16)
        'avg_in_amount': float(avg_in_amt),                                     # (17)
        'avg_out_amount': float(avg_out_amt),                                   # (18)
        'total_in_gas_used': int(total_in_gas),                                 # (19)
        'total_out_gas_used': int(total_out_gas),                               # (20)
        'avg_in_gas_used': float(avg_in_gas),                                   # (21)
        'avg_out_gas_used': float(avg_out_gas),                                 # (22)

        # 23~25 일 단위
        'active_days': int(active_days),                                        # (23)
        'tx_per_day': float(tx_per_day),                                        # (24)
        'amount_per_day': float(amt_per_day),                                   # (25)

        # 26~27 가스 효율
        'recv_gas_efficiency': float(recv_gas_eff),                             # (26)
        'send_gas_efficiency': float(send_gas_eff),                             # (27)

        # 28~29 첫/마지막 거래시각 (epoch seconds)
        'first_tx_time': int(first_ts) if first_ts is not None else None,       # (28)
        'last_tx_time': int(last_ts) if last_ts is not None else None           # (29)
    })

node_features = pd.DataFrame(rows).sort_values('address').reset_index(drop=True)


In [6]:
# 4) 산출물 저장
node_features.to_csv('node_features.csv', index=False, encoding='utf-8')

import pickle
with open("G_base_multidigraph.pkl", "wb") as f:
    pickle.dump(G_base, f)
with open("H_summary_graph.pkl", "wb") as f:
    pickle.dump(H_summary, f)

print("✅ 완료: node_features.csv / G_base_multidigraph.pkl / H_summary_graph.pkl 생성")

✅ 완료: node_features.csv / G_base_multidigraph.pkl / H_summary_graph.pkl 생성


### Node2vec방식으로 그래프구조만 벡터로 임베딩하여 학습 - GAT알고리즘

In [2]:
# -*- coding: utf-8 -*-
"""
Node2Vec(64d; CPU, 저장/재사용) + GAT 오토인코더(256->32->64; DEVICE)
→ 재구성오차 z-score → 상위 5% 이상치

- Node2Vec은 CPU에서만 수행 (edge_index도 CPU)
- DataLoader는 num_workers=0 (Windows/Jupyter PyCapsule 피클링 에러 회피)
- 임베딩/엣지/주소를 디스크에 저장하여 다음 세션에서 재사용
"""

import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import Node2Vec, GATConv
from torch_geometric.data import Data

# ------------------------------------------------------------
# 경로/환경
NODE_FEAT_CSV = "node_features.csv"
GRAPH_PKL     = "G_base_multidigraph.pkl"

EMB_NPY = "node2vec_embeddings.npy"  # 저장될 임베딩
EDGE_PT = "edge_index.pt"            # 저장될 edge_index(CPU long)
ADDR_CSV = "addresses.csv"           # 저장될 주소 순서
N2V_PT  = "node2vec.pt"              # (선택) Node2Vec state_dict

FORCE_RETRAIN = False  # True로 바꾸면 임베딩을 강제로 다시 학습

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

# ------------------------------------------------------------
# 0) 노드/그래프 로드
if not os.path.exists(NODE_FEAT_CSV):
    raise FileNotFoundError(f"{NODE_FEAT_CSV} 파일이 없습니다.")
if not os.path.exists(GRAPH_PKL):
    raise FileNotFoundError(f"{GRAPH_PKL} 파일이 없습니다.")

node_df = pd.read_csv(NODE_FEAT_CSV)
addresses_cur = node_df['address'].astype(str).tolist()
addr2idx = {a: i for i, a in enumerate(addresses_cur)}

with open(GRAPH_PKL, "rb") as f:
    G_nx = pickle.load(f)  # networkx.MultiDiGraph

# edge_index (CPU 텐서)
edges = []
for u, v, _k in G_nx.edges(keys=True):
    if u in addr2idx and v in addr2idx:
        edges.append([addr2idx[u], addr2idx[v]])
edge_index_cpu = torch.tensor(edges, dtype=torch.long).t().contiguous()  # [2,E] (CPU)

num_nodes = len(addresses_cur)
num_edges = edge_index_cpu.size(1)
print(f"DEVICE: {DEVICE} | nodes: {num_nodes} | edges: {num_edges}")

# (선택) 양방향 추가: 안정성/성능 향상에 도움
# edge_index_cpu = torch.cat([edge_index_cpu, edge_index_cpu.flip(0)], dim=1)

# ------------------------------------------------------------
# 1) Node2Vec 임베딩: 저장본이 있으면 로드, 없으면 학습 후 저장
use_saved = (not FORCE_RETRAIN) and os.path.exists(EMB_NPY) and os.path.exists(EDGE_PT) and os.path.exists(ADDR_CSV)

def load_saved_embeddings():
    # 주소 순서 일치 여부 검증
    addresses_saved = pd.read_csv(ADDR_CSV)['address'].astype(str).tolist()
    if len(addresses_saved) != len(addresses_cur) or any(a!=b for a,b in zip(addresses_saved, addresses_cur)):
        print("⚠️ 저장된 addresses.csv 와 현재 addresses 순서/개수가 다릅니다. 재학습으로 전환합니다.")
        return None, None, None

    x_np = np.load(EMB_NPY)  # (N,64)
    if x_np.shape[0] != len(addresses_cur) or x_np.shape[1] != 64:
        print("⚠️ 저장된 임베딩 크기가 예상과 다릅니다. 재학습으로 전환합니다.")
        return None, None, None

    edge_idx_saved = torch.load(EDGE_PT, map_location="cpu")
    # 간단 검증: dtype/shape
    if edge_idx_saved.dtype != torch.long or edge_idx_saved.dim()!=2 or edge_idx_saved.size(0)!=2:
        print("⚠️ 저장된 edge_index 형식이 예상과 다릅니다. 재학습으로 전환합니다.")
        return None, None, None

    print("✅ 저장된 Node2Vec 임베딩/엣지/주소를 불러옵니다.")
    return x_np, edge_idx_saved, addresses_saved

x_init_cpu = None
if use_saved:
    x_np, edge_index_loaded, addresses_saved = load_saved_embeddings()
    if x_np is not None:
        x_init_cpu = torch.tensor(x_np, dtype=torch.float32)  # (N,64) CPU
        edge_index_cpu = edge_index_loaded                    # 저장된 엣지로 교체(안전)
        addresses = addresses_saved
    else:
        use_saved = False  # 불일치 -> 재학습

if not use_saved:
    # Node2Vec (CPU에서만 수행)
    print("🛠️ Node2Vec 임베딩을 새로 학습합니다...")
    data_cpu = Data(edge_index=edge_index_cpu, num_nodes=num_nodes)  # CPU 전용 Data

    n2v = Node2Vec(
        data_cpu.edge_index,  # CPU 텐서
        embedding_dim=64,
        walk_length=30,
        context_size=10,     # window
        walks_per_node=200,  # num_walk
        p=1.0, q=1.0,
        num_negative_samples=1,
        sparse=True          # SparseAdam 사용
    )
    n2v_loader = n2v.loader(batch_size=128, shuffle=True, num_workers=0, persistent_workers=False)
    n2v_optimizer = torch.optim.SparseAdam(list(n2v.parameters()), lr=0.01)

    def train_node2vec(epochs=5):
        n2v.train()
        for epoch in range(1, epochs + 1):
            total_loss = 0.0
            for pos_rw, neg_rw in n2v_loader:
                n2v_optimizer.zero_grad()
                loss = n2v.loss(pos_rw, neg_rw)  # CPU 경로
                loss.backward()
                n2v_optimizer.step()
                total_loss += loss.item()
            print(f"[Node2Vec] epoch {epoch:03d} | loss {total_loss/len(n2v_loader):.4f}")

    train_node2vec(epochs=5)

    with torch.no_grad():
        x_init_cpu = n2v.embedding.weight.clone().detach()     # (N,64) CPU

    # ─ 저장 ─
    np.save(EMB_NPY, x_init_cpu.numpy())
    torch.save(edge_index_cpu, EDGE_PT)
    pd.Series(addresses_cur, name="address").to_csv(ADDR_CSV, index=False)
    try:
        torch.save(n2v.state_dict(), N2V_PT)
    except Exception as e:
        print(f"(참고) Node2Vec state 저장 생략: {e}")
    addresses = addresses_cur
    print(f"✅ 저장 완료: {EMB_NPY}, {EDGE_PT}, {ADDR_CSV}, {N2V_PT}")

# 최종 입력 텐서 (DEVICE로 이동)
x_in = x_init_cpu.to(DEVICE)
edge_index_dev = edge_index_cpu.to(DEVICE)

# ------------------------------------------------------------
# 2) GAT 오토인코더 (DEVICE에서 수행)
class GATAutoEncoder(nn.Module):
    def __init__(self, in_dim=64, dropout=0.3):
        super().__init__()
        self.dropout = dropout
        # 64 -> (32 * 8) = 256
        self.gat1 = GATConv(in_channels=in_dim, out_channels=32, heads=8, concat=True, dropout=dropout)
        # 256 -> 32
        self.gat2 = GATConv(in_channels=256, out_channels=32, heads=1, concat=True, dropout=dropout)
        # 32 -> 64
        self.gat3 = GATConv(in_channels=32, out_channels=64, heads=1, concat=True, dropout=dropout)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gat1(x, edge_index)
        x = F.elu(x)

        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gat2(x, edge_index)
        x = F.elu(x)

        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gat3(x, edge_index)  # 최종 복원 64d
        return x

model = GATAutoEncoder(in_dim=64, dropout=0.3).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)

# ------------------------------------------------------------
# 3) 학습 (1000 epochs, MSE 재구성오차)
def train_gat_ae(epochs=1000):
    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        x_hat = model(x_in, edge_index_dev)
        loss = F.mse_loss(x_hat, x_in)
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0 or epoch == 1:
            print(f"[GAT-AE] epoch {epoch:04d} | recon MSE {loss.item():.6f}")

train_gat_ae(epochs=1000)

# ------------------------------------------------------------
# 4) 재구성오차 → z-score → 상위 5% 이상치
model.eval()
with torch.no_grad():
    x_recon = model(x_in, edge_index_dev)

recon_err = torch.mean((x_recon - x_in) ** 2, dim=1).detach().cpu().numpy()
mu = recon_err.mean()
sigma = recon_err.std(ddof=1) if recon_err.size > 1 else 1e-8
z = (recon_err - mu) / (sigma if sigma > 0 else 1e-8)
z_cut = np.percentile(z, 95.0)
anom = (z > z_cut).astype(int)

print(f"상위 5% z-score 임계값: {z_cut:.4f}")
print(f"이상치 노드 수: {anom.sum()} / {len(anom)}")

# ------------------------------------------------------------
# 5) 결과 저장
out = pd.DataFrame({
    "address": addresses,
    "recon_mse": recon_err,
    "z_score": z,
    "is_anomaly_top5pct": anom
})
out.sort_values("z_score", ascending=False).to_csv("gat_ae_anomalies.csv", index=False, encoding="utf-8")
torch.save(model.state_dict(), "gat_autoencoder.pt")

print("✅ 완료: gat_ae_anomalies.csv / gat_autoencoder.pt 생성")
print(f"ℹ️ 임베딩/엣지는 파일로 보관됨 → {EMB_NPY}, {EDGE_PT}, {ADDR_CSV} (필요 시 재사용)")

DEVICE: cpu | nodes: 7958 | edges: 25601
🛠️ Node2Vec 임베딩을 새로 학습합니다...
[Node2Vec] epoch 001 | loss 3.0723
[Node2Vec] epoch 002 | loss 1.3729
[Node2Vec] epoch 003 | loss 0.9691
[Node2Vec] epoch 004 | loss 0.8544
[Node2Vec] epoch 005 | loss 0.8131
✅ 저장 완료: node2vec_embeddings.npy, edge_index.pt, addresses.csv, node2vec.pt
[GAT-AE] epoch 0001 | recon MSE 0.251541
[GAT-AE] epoch 0050 | recon MSE 0.093097
[GAT-AE] epoch 0100 | recon MSE 0.075579
[GAT-AE] epoch 0150 | recon MSE 0.072817
[GAT-AE] epoch 0200 | recon MSE 0.073885
[GAT-AE] epoch 0250 | recon MSE 0.069703
[GAT-AE] epoch 0300 | recon MSE 0.067430
[GAT-AE] epoch 0350 | recon MSE 0.066992
[GAT-AE] epoch 0400 | recon MSE 0.066373
[GAT-AE] epoch 0450 | recon MSE 0.065949
[GAT-AE] epoch 0500 | recon MSE 0.066416
[GAT-AE] epoch 0550 | recon MSE 0.065492
[GAT-AE] epoch 0600 | recon MSE 0.065401
[GAT-AE] epoch 0650 | recon MSE 0.065288
[GAT-AE] epoch 0700 | recon MSE 0.065069
[GAT-AE] epoch 0750 | recon MSE 0.065271
[GAT-AE] epoch 0800 | r

### Node2vec방식으로 그래프구조만 벡터로 임베딩하여 학습 - GraphSAGE알고리즘

In [3]:
# -*- coding: utf-8 -*-
"""
Node2Vec(64d; CPU, 저장/재사용) + GraphSAGE 오토인코더(256->32->64; DEVICE)
→ 재구성오차 z-score → 상위 5% 이상치

- Node2Vec은 CPU에서만 수행 (edge_index도 CPU)
- DataLoader num_workers=0 (Windows/Jupyter PyCapsule 피클링 에러 회피)
- 임베딩/엣지/주소를 디스크에 저장하여 다음 세션에서 재사용
"""

import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import Node2Vec, SAGEConv
from torch_geometric.data import Data

# ------------------------------------------------------------
# 경로/환경
NODE_FEAT_CSV = "node_features.csv"
GRAPH_PKL     = "G_base_multidigraph.pkl"

# Node2Vec 산출물 (GAT 스크립트와 공유 가능)
EMB_NPY  = "node2vec_embeddings.npy"  # 저장될 임베딩 (N,64)
EDGE_PT  = "edge_index.pt"            # 저장될 edge_index (CPU long)
ADDR_CSV = "addresses.csv"            # 저장될 주소 순서
N2V_PT   = "node2vec.pt"              # (선택) Node2Vec state_dict

FORCE_RETRAIN = False  # True면 Node2Vec을 강제로 다시 학습

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

# ------------------------------------------------------------
# 0) 노드/그래프 로드
if not os.path.exists(NODE_FEAT_CSV):
    raise FileNotFoundError(f"{NODE_FEAT_CSV} 파일이 없습니다.")
if not os.path.exists(GRAPH_PKL):
    raise FileNotFoundError(f"{GRAPH_PKL} 파일이 없습니다.")

node_df = pd.read_csv(NODE_FEAT_CSV)
addresses_cur = node_df['address'].astype(str).tolist()
addr2idx = {a: i for i, a in enumerate(addresses_cur)}

with open(GRAPH_PKL, "rb") as f:
    G_nx = pickle.load(f)  # networkx.MultiDiGraph

# edge_index (CPU 텐서)
edges = []
for u, v, _k in G_nx.edges(keys=True):
    if u in addr2idx and v in addr2idx:
        edges.append([addr2idx[u], addr2idx[v]])
edge_index_cpu = torch.tensor(edges, dtype=torch.long).t().contiguous()  # [2,E] (CPU)

num_nodes = len(addresses_cur)
num_edges = edge_index_cpu.size(1)
print(f"DEVICE: {DEVICE} | nodes: {num_nodes} | edges: {num_edges}")

# (선택) 무방향 효과 추가
# edge_index_cpu = torch.cat([edge_index_cpu, edge_index_cpu.flip(0)], dim=1)

# ------------------------------------------------------------
# 1) Node2Vec 임베딩: 저장본이 있으면 로드, 없으면 학습 후 저장
use_saved = (not FORCE_RETRAIN) and os.path.exists(EMB_NPY) and os.path.exists(EDGE_PT) and os.path.exists(ADDR_CSV)

def load_saved_embeddings():
    addresses_saved = pd.read_csv(ADDR_CSV)['address'].astype(str).tolist()
    if len(addresses_saved) != len(addresses_cur) or any(a!=b for a,b in zip(addresses_saved, addresses_cur)):
        print("⚠️ 저장된 addresses.csv 와 현재 addresses 순서/개수가 다릅니다. 재학습으로 전환합니다.")
        return None, None, None

    x_np = np.load(EMB_NPY)  # (N,64)
    if x_np.shape != (len(addresses_cur), 64):
        print("⚠️ 저장된 임베딩 크기가 예상과 다릅니다. 재학습으로 전환합니다.")
        return None, None, None

    edge_idx_saved = torch.load(EDGE_PT, map_location="cpu")
    if edge_idx_saved.dtype != torch.long or edge_idx_saved.dim()!=2 or edge_idx_saved.size(0)!=2:
        print("⚠️ 저장된 edge_index 형식이 예상과 다릅니다. 재학습으로 전환합니다.")
        return None, None, None

    print("✅ 저장된 Node2Vec 임베딩/엣지/주소를 불러옵니다.")
    return x_np, edge_idx_saved, addresses_saved

x_init_cpu = None
if use_saved:
    x_np, edge_index_loaded, addresses_saved = load_saved_embeddings()
    if x_np is not None:
        x_init_cpu = torch.tensor(x_np, dtype=torch.float32)  # (N,64) CPU
        edge_index_cpu = edge_index_loaded
        addresses = addresses_saved
    else:
        use_saved = False

if not use_saved:
    print("🛠️ Node2Vec 임베딩을 새로 학습합니다...")
    data_cpu = Data(edge_index=edge_index_cpu, num_nodes=num_nodes)

    n2v = Node2Vec(
        data_cpu.edge_index,
        embedding_dim=64,
        walk_length=30,
        context_size=10,     # window
        walks_per_node=200,  # num_walk
        p=1.0, q=1.0,
        num_negative_samples=1,
        sparse=True
    )
    n2v_loader = n2v.loader(batch_size=128, shuffle=True, num_workers=0, persistent_workers=False)
    n2v_optimizer = torch.optim.SparseAdam(list(n2v.parameters()), lr=0.01)

    def train_node2vec(epochs=5):
        n2v.train()
        for epoch in range(1, epochs + 1):
            total_loss = 0.0
            for pos_rw, neg_rw in n2v_loader:
                n2v_optimizer.zero_grad()
                loss = n2v.loss(pos_rw, neg_rw)
                loss.backward()
                n2v_optimizer.step()
                total_loss += loss.item()
            print(f"[Node2Vec] epoch {epoch:03d} | loss {total_loss/len(n2v_loader):.4f}")

    train_node2vec(epochs=5)

    with torch.no_grad():
        x_init_cpu = n2v.embedding.weight.clone().detach()  # (N,64) CPU

    # 저장
    np.save(EMB_NPY, x_init_cpu.numpy())
    torch.save(edge_index_cpu, EDGE_PT)
    pd.Series(addresses_cur, name="address").to_csv(ADDR_CSV, index=False)
    try:
        torch.save(n2v.state_dict(), N2V_PT)
    except Exception as e:
        print(f"(참고) Node2Vec state 저장 생략: {e}")
    addresses = addresses_cur
    print(f"✅ 저장 완료: {EMB_NPY}, {EDGE_PT}, {ADDR_CSV}, {N2V_PT}")

# 최종 입력/엣지 (DEVICE로 이동)
x_in = x_init_cpu.to(DEVICE)          # (N,64)
edge_index_dev = edge_index_cpu.to(DEVICE)

# ------------------------------------------------------------
# 2) GraphSAGE 오토인코더 (DEVICE에서 수행)
class SAGEAutoEncoder(nn.Module):
    def __init__(self, in_dim=64, dropout=0.3, aggr="mean"):
        super().__init__()
        self.dropout = dropout
        self.s1 = SAGEConv(in_dim, 256, aggr=aggr)  # 64 -> 256
        self.s2 = SAGEConv(256, 32, aggr=aggr)      # 256 -> 32
        self.s3 = SAGEConv(32, 64, aggr=aggr)       # 32 -> 64
    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training); x = F.elu(self.s1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training); x = F.elu(self.s2(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training); x = self.s3(x, edge_index)
        return x

model = SAGEAutoEncoder(in_dim=64, dropout=0.3, aggr="mean").to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)

# ------------------------------------------------------------
# 3) 학습 (1000 epochs, MSE 재구성오차)
def train_sage_ae(epochs=1000):
    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        x_hat = model(x_in, edge_index_dev)
        loss = F.mse_loss(x_hat, x_in)
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0 or epoch == 1:
            print(f"[SAGE-AE] epoch {epoch:04d} | recon MSE {loss.item():.6f}")

train_sage_ae(epochs=1000)

# ------------------------------------------------------------
# 4) 재구성오차 → z-score → 상위 5% 이상치
model.eval()
with torch.no_grad():
    x_recon = model(x_in, edge_index_dev)

recon_err = torch.mean((x_recon - x_in) ** 2, dim=1).detach().cpu().numpy()
mu = recon_err.mean()
sigma = recon_err.std(ddof=1) if recon_err.size > 1 else 1e-8
z = (recon_err - mu) / (sigma if sigma > 0 else 1e-8)
z_cut = np.percentile(z, 95.0)
anom = (z > z_cut).astype(int)

print(f"상위 5% z-score 임계값: {z_cut:.4f}")
print(f"이상치 노드 수: {anom.sum()} / {len(anom)}")

# ------------------------------------------------------------
# 5) 결과 저장 (SAGE 파일명)
out = pd.DataFrame({
    "address": addresses,
    "recon_mse": recon_err,
    "z_score": z,
    "is_anomaly_top5pct": anom
})
out.sort_values("z_score", ascending=False).to_csv("sage_ae_anomalies.csv", index=False, encoding="utf-8")
torch.save(model.state_dict(), "sage_autoencoder.pt")

print("✅ 완료: sage_ae_anomalies.csv / sage_autoencoder.pt 생성")
print(f"ℹ️ 임베딩/엣지는 파일로 보관됨 → {EMB_NPY}, {EDGE_PT}, {ADDR_CSV} (필요 시 재사용)")

DEVICE: cpu | nodes: 7958 | edges: 25601
✅ 저장된 Node2Vec 임베딩/엣지/주소를 불러옵니다.
[SAGE-AE] epoch 0001 | recon MSE 0.250588


C:\Users\user\AppData\Local\Temp\ipykernel_30980\1176852788.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  edge_idx_saved = torch.load(EDGE_PT, map_location="cpu")


[SAGE-AE] epoch 0050 | recon MSE 0.130431
[SAGE-AE] epoch 0100 | recon MSE 0.090441
[SAGE-AE] epoch 0150 | recon MSE 0.085147
[SAGE-AE] epoch 0200 | recon MSE 0.074807
[SAGE-AE] epoch 0250 | recon MSE 0.068855
[SAGE-AE] epoch 0300 | recon MSE 0.067581
[SAGE-AE] epoch 0350 | recon MSE 0.067303
[SAGE-AE] epoch 0400 | recon MSE 0.064358
[SAGE-AE] epoch 0450 | recon MSE 0.063148
[SAGE-AE] epoch 0500 | recon MSE 0.063130
[SAGE-AE] epoch 0550 | recon MSE 0.062990
[SAGE-AE] epoch 0600 | recon MSE 0.062119
[SAGE-AE] epoch 0650 | recon MSE 0.061254
[SAGE-AE] epoch 0700 | recon MSE 0.060762
[SAGE-AE] epoch 0750 | recon MSE 0.060542
[SAGE-AE] epoch 0800 | recon MSE 0.061159
[SAGE-AE] epoch 0850 | recon MSE 0.059888
[SAGE-AE] epoch 0900 | recon MSE 0.059822
[SAGE-AE] epoch 0950 | recon MSE 0.059361
[SAGE-AE] epoch 1000 | recon MSE 0.059117
상위 5% z-score 임계값: 1.1954
이상치 노드 수: 398 / 7958
✅ 완료: sage_ae_anomalies.csv / sage_autoencoder.pt 생성
ℹ️ 임베딩/엣지는 파일로 보관됨 → node2vec_embeddings.npy, edge_index.pt,

### Node2vec으로 그래프구조임베딩값과 29개의 node피쳐를 결합한 64차원의 그래프임베딩벡터 생성후 GAT,GraphSGAE알고리즘 학습 진행

In [4]:
# -*- coding: utf-8 -*-
"""
Node2Vec(64, 저장/재사용) + 29속성 결합 → 64차원 입력 생성(PCA)
→ GAT-GAE & SAGE-GAE 모두 학습
- 특징 재구성(MSE) + 링크 재구성(BCE) 동시 학습
"""

import os
import pickle
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import Node2Vec, GATConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.utils import negative_sampling
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from joblib import dump, load

# ----------------------------- 설정 ------------------------------
NODE_FEAT_CSV = "node_features.csv"
GRAPH_PKL     = "G_base_multidigraph.pkl"

# Node2Vec 산출물
EMB_NPY  = "node2vec_embeddings.npy"
EDGE_PT  = "edge_index.pt"
ADDR_CSV = "addresses.csv"
N2V_PT   = "node2vec.pt"

# 결합 임베딩/전처리 저장
COMB_EMB_NPY = "combined_embeddings_64.npy"
SCALER_P     = "scaler.joblib"
PCA64_P      = "pca64.joblib"

FORCE_RETRAIN_N2V = False      # True면 Node2Vec 재학습 강제
LAMBDA_BCE        = 0.5        # 총손실 = MSE + λ*BCE
EPOCHS            = 1000
DROPOUT           = 0.3
LR                = 1e-3
WEIGHT_DECAY      = 5e-4
SEED              = 42

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED); np.random.seed(SEED)

# ---------------------- 0) 주소/그래프 로드 -----------------------
if not os.path.exists(NODE_FEAT_CSV):
    raise FileNotFoundError(f"{NODE_FEAT_CSV} 없음")
if not os.path.exists(GRAPH_PKL):
    raise FileNotFoundError(f"{GRAPH_PKL} 없음")

node_df = pd.read_csv(NODE_FEAT_CSV)
addresses_cur = node_df['address'].astype(str).tolist()
addr2idx = {a: i for i,a in enumerate(addresses_cur)}

with open(GRAPH_PKL, "rb") as f:
    import networkx as nx
    G_nx = pickle.load(f)  # MultiDiGraph

# edge_index (CPU)
edges = []
for u, v, _k in G_nx.edges(keys=True):
    if u in addr2idx and v in addr2idx:
        edges.append([addr2idx[u], addr2idx[v]])
edge_index_cpu = torch.tensor(edges, dtype=torch.long).t().contiguous()

num_nodes = len(addresses_cur)
print(f"DEVICE: {DEVICE} | nodes: {num_nodes} | edges: {edge_index_cpu.size(1)}")

# (선택) 양방향 추가
# edge_index_cpu = torch.cat([edge_index_cpu, edge_index_cpu.flip(0)], dim=1)

# ---------------------- 1) Node2Vec 로드/학습 ----------------------
def try_load_saved_n2v():
    if not (os.path.exists(EMB_NPY) and os.path.exists(EDGE_PT) and os.path.exists(ADDR_CSV)):
        return None
    addr_saved = pd.read_csv(ADDR_CSV)['address'].astype(str).tolist()
    if len(addr_saved)!=len(addresses_cur) or any(a!=b for a,b in zip(addr_saved, addresses_cur)):
        print("⚠️ 저장된 addresses와 현재가 불일치 → N2V 재학습")
        return None
    x_np = np.load(EMB_NPY)
    if x_np.shape!=(len(addresses_cur), 64):
        print("⚠️ 저장된 임베딩 크기 불일치 → N2V 재학습")
        return None
    edge_saved = torch.load(EDGE_PT, map_location="cpu")
    if edge_saved.dtype!=torch.long or edge_saved.dim()!=2 or edge_saved.size(0)!=2:
        print("⚠️ 저장된 edge_index 형식 이상 → N2V 재학습")
        return None
    print("✅ 저장된 Node2Vec 임베딩/엣지/주소 사용")
    return torch.tensor(x_np, dtype=torch.float32), edge_saved, addr_saved

x_n2v_cpu = None
if not FORCE_RETRAIN_N2V:
    loaded = try_load_saved_n2v()
    if loaded:
        x_n2v_cpu, edge_index_cpu, addresses = loaded

if x_n2v_cpu is None:
    print("🛠️ Node2Vec를 새로 학습합니다...")
    data_cpu = Data(edge_index=edge_index_cpu, num_nodes=num_nodes)
    n2v = Node2Vec(
        data_cpu.edge_index, embedding_dim=64,
        walk_length=30, context_size=10, walks_per_node=200,
        p=1.0, q=1.0, num_negative_samples=1, sparse=True
    )
    n2v_loader = n2v.loader(batch_size=128, shuffle=True, num_workers=0, persistent_workers=False)
    n2v_opt = torch.optim.SparseAdam(list(n2v.parameters()), lr=0.01)

    def train_n2v(epochs=5):
        n2v.train()
        for ep in range(1, epochs+1):
            tot=0.0
            for pos_rw, neg_rw in n2v_loader:
                n2v_opt.zero_grad()
                loss = n2v.loss(pos_rw, neg_rw)
                loss.backward(); n2v_opt.step()
                tot += loss.item()
            print(f"[Node2Vec] epoch {ep:03d} | loss {tot/len(n2v_loader):.4f}")

    train_n2v(epochs=5)
    with torch.no_grad():
        x_n2v_cpu = n2v.embedding.weight.clone().detach()   # (N,64)
    # 저장
    np.save(EMB_NPY, x_n2v_cpu.numpy())
    torch.save(edge_index_cpu, EDGE_PT)
    pd.Series(addresses_cur, name="address").to_csv(ADDR_CSV, index=False)
    try: torch.save(n2v.state_dict(), N2V_PT)
    except Exception as e: print(f"(참고) Node2Vec state 저장 생략: {e}")
    addresses = addresses_cur
    print("✅ Node2Vec 저장 완료")

# ---------------- 2) 29속성과 결합 → 64차 투영(PCA) ----------------
node = pd.read_csv(NODE_FEAT_CSV).set_index("address")
feat_cols = [c for c in node.columns]  # 주소 제외 29개
X_attr = node.reindex(addresses)[feat_cols].replace([np.inf,-np.inf], np.nan).fillna(0).astype(float).values  # (N,29)

# 표준화
if os.path.exists(SCALER_P) and os.path.exists(PCA64_P):
    try:
        scaler = load(SCALER_P); pca64 = load(PCA64_P)
        print("✅ 저장된 Scaler/PCA 로드")
    except Exception:
        scaler = StandardScaler().fit(X_attr)
        pca64  = PCA(n_components=64, random_state=SEED).fit(np.hstack([x_n2v_cpu.numpy(), scaler.transform(X_attr)]))
        dump(scaler, SCALER_P); dump(pca64, PCA64_P)
else:
    scaler = StandardScaler().fit(X_attr)
    dump(scaler, SCALER_P)
    # concat(64+29=93) → PCA 64
    X_concat_fit = np.hstack([x_n2v_cpu.numpy(), scaler.transform(X_attr)])
    pca64 = PCA(n_components=64, random_state=SEED).fit(X_concat_fit)
    dump(pca64, PCA64_P)

X_concat = np.hstack([x_n2v_cpu.numpy(), scaler.transform(X_attr)])   # (N,93)
X64 = pca64.transform(X_concat).astype(np.float32)                     # (N,64)
np.save(COMB_EMB_NPY, X64)
print("✅ 결합 임베딩 저장:", COMB_EMB_NPY)

x_in = torch.tensor(X64, dtype=torch.float32, device=DEVICE)
edge_index_dev = edge_index_cpu.to(DEVICE)

# ----------------------- 3) 모델 정의 ------------------------
class GAT_GAE(nn.Module):
    def __init__(self, in_dim=64, dropout=0.3):
        super().__init__()
        self.do = dropout
        self.g1 = GATConv(in_dim, 32, heads=8, concat=True, dropout=dropout)  # 64 -> 256
        self.g2 = GATConv(256, 32, heads=1, concat=True, dropout=dropout)     # 256 -> 32 (z)
        self.g3 = GATConv(32, 64, heads=1, concat=True, dropout=dropout)      # 32 -> 64 (x_hat)
    def forward(self, x, ei):
        x = F.dropout(x, p=self.do, training=self.training); x = F.elu(self.g1(x, ei))
        x = F.dropout(x, p=self.do, training=self.training); z = F.elu(self.g2(x, ei))
        x_hat = F.dropout(z, p=self.do, training=self.training); x_hat = self.g3(x_hat, ei)
        return x_hat, z

class SAGE_GAE(nn.Module):
    def __init__(self, in_dim=64, dropout=0.3, aggr="mean"):
        super().__init__()
        self.do = dropout
        self.s1 = SAGEConv(in_dim, 256, aggr=aggr)
        self.s2 = SAGEConv(256, 32, aggr=aggr)   # z
        self.s3 = SAGEConv(32, 64, aggr=aggr)    # x_hat
    def forward(self, x, ei):
        x = F.dropout(x, p=self.do, training=self.training); x = F.elu(self.s1(x, ei))
        z = F.dropout(x, p=self.do, training=self.training); z = F.elu(self.s2(z, ei))
        x_hat = F.dropout(z, p=self.do, training=self.training); x_hat = self.s3(x_hat, ei)
        return x_hat, z

def link_bce_loss(z, edge_index, num_neg=None):
    # pos/neg 샘플
    pos = edge_index
    if num_neg is None: num_neg = pos.size(1)
    neg = negative_sampling(pos, num_nodes=z.size(0), num_neg_samples=num_neg, method='sparse')
    # dot decode
    def dot_decode(z, e): return (z[e[0]] * z[e[1]]).sum(dim=1)
    logits = torch.cat([dot_decode(z, pos), dot_decode(z, neg)], dim=0)
    labels = torch.cat([
        torch.ones(pos.size(1), device=logits.device),
        torch.zeros(neg.size(1), device=logits.device)
    ], dim=0)
    return F.binary_cross_entropy_with_logits(logits, labels)

def train_gae(model, epochs, name):
    model = model.to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    for ep in range(1, epochs+1):
        model.train(); opt.zero_grad()
        x_hat, z = model(x_in, edge_index_dev)
        mse = F.mse_loss(x_hat, x_in)
        bce = link_bce_loss(z, edge_index_dev)
        loss = mse + LAMBDA_BCE * bce
        loss.backward(); opt.step()
        if ep==1 or ep%50==0:
            print(f"[{name}] {ep:04d} | loss {loss.item():.6f} | mse {mse.item():.6f} | bce {bce.item():.6f}")
    # 평가/저장
    model.eval()
    with torch.no_grad():
        x_hat, z = model(x_in, edge_index_dev)
    recon = torch.mean((x_hat - x_in)**2, dim=1).detach().cpu().numpy()
    mu, sd = recon.mean(), recon.std(ddof=1) if recon.size>1 else 1e-8
    zscore = (recon - mu) / (sd if sd>0 else 1e-8)
    cut = np.percentile(zscore, 95.0)
    anom = (zscore > cut).astype(int)

    out = pd.DataFrame({"address": addresses, "recon_mse": recon, "z_score": zscore, "is_anomaly_top5pct": anom})
    csv_name = f"{name.lower()}_anomalies.csv"
    pt_name  = f"{name.lower()}_model.pt"
    out.sort_values("z_score", ascending=False).to_csv(csv_name, index=False, encoding="utf-8")
    torch.save(model.state_dict(), pt_name)
    print(f"✅ {name}: 저장 완료 → {csv_name}, {pt_name}")
    return out

# -------------------- 4) 두 모델 연속 실행 --------------------
gat_out  = train_gae(GAT_GAE(in_dim=64, dropout=DROPOUT),  EPOCHS, "GAT_GAE")
sage_out = train_gae(SAGE_GAE(in_dim=64, dropout=DROPOUT), EPOCHS, "SAGE_GAE")

print("✅ 완료: combined_embeddings_64.npy / scaler.joblib / pca64.joblib 저장")
print("ℹ️ GAT_GAE → gat_gae_anomalies.csv, SAGE_GAE → sage_gae_anomalies.csv")

DEVICE: cpu | nodes: 7958 | edges: 25601
✅ 저장된 Node2Vec 임베딩/엣지/주소 사용
✅ 결합 임베딩 저장: combined_embeddings_64.npy
[GAT_GAE] 0001 | loss 505.491791 | mse 108.071953 | bce 794.839661


C:\Users\user\AppData\Local\Temp\ipykernel_30980\3738493665.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  edge_saved = torch.load(EDGE_PT, map_location="cpu")


[GAT_GAE] 0050 | loss 8.589891 | mse 2.541967 | bce 12.095848
[GAT_GAE] 0100 | loss 3.123414 | mse 1.463450 | bce 3.319929
[GAT_GAE] 0150 | loss 2.923254 | mse 1.345032 | bce 3.156443
[GAT_GAE] 0200 | loss 2.850860 | mse 1.092607 | bce 3.516506
[GAT_GAE] 0250 | loss 3.365918 | mse 1.056040 | bce 4.619755
[GAT_GAE] 0300 | loss 2.799266 | mse 1.162821 | bce 3.272889
[GAT_GAE] 0350 | loss 2.319847 | mse 1.054340 | bce 2.531014
[GAT_GAE] 0400 | loss 2.057616 | mse 0.873196 | bce 2.368840
[GAT_GAE] 0450 | loss 1.591910 | mse 0.789524 | bce 1.604770
[GAT_GAE] 0500 | loss 1.530125 | mse 0.826689 | bce 1.406873
[GAT_GAE] 0550 | loss 1.718401 | mse 0.818809 | bce 1.799184
[GAT_GAE] 0600 | loss 1.560501 | mse 0.835535 | bce 1.449933
[GAT_GAE] 0650 | loss 1.142637 | mse 0.689243 | bce 0.906788
[GAT_GAE] 0700 | loss 1.374990 | mse 0.669408 | bce 1.411165
[GAT_GAE] 0750 | loss 1.072892 | mse 0.648325 | bce 0.849134
[GAT_GAE] 0800 | loss 0.996120 | mse 0.624989 | bce 0.742264
[GAT_GAE] 0850 | loss 0

### GAT,GraphSAGE 모두 미니배치방식으로 학습한 방식

In [5]:
# -*- coding: utf-8 -*-
"""
미니배치(Neighbor Sampling)로 학습하는 GAT-GAE & SAGE-GAE
입력: Node2Vec(64) + 29속성 결합 → PCA 64 (저장/재사용)
손실: 특징 재구성(MSE) + 링크 재구성(BCE, seed 내부 링크)
평가도 미니배치로 seed별 x_hat을 붙여 전체 recon MSE 계산
"""

import os, pickle
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import Node2Vec, GATConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import NeighborLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from joblib import dump, load

# ----------------------------- 설정 ------------------------------
NODE_FEAT_CSV = "node_features.csv"
GRAPH_PKL     = "G_base_multidigraph.pkl"

# Node2Vec 산출물
EMB_NPY  = "node2vec_embeddings.npy"
EDGE_PT  = "edge_index.pt"
ADDR_CSV = "addresses.csv"
N2V_PT   = "node2vec.pt"

# 결합 임베딩/전처리 저장
COMB_EMB_NPY = "combined_embeddings_64.npy"
SCALER_P     = "scaler.joblib"
PCA64_P      = "pca64.joblib"

# 미니배치 하이퍼파라미터
BATCH_SIZE   = 1024        # 노드 seed 배치 크기
FANOUTS      = [15, 10, 5] # 레이어별 이웃 샘플 크기(=num_neighbors)
NUM_WORKERS  = 0           # Windows/Jupyter 안전값
PERSISTENT   = False

FORCE_RETRAIN_N2V = False
LAMBDA_BCE        = 0.5
EPOCHS            = 1000
DROPOUT           = 0.3
LR                = 1e-3
WEIGHT_DECAY      = 5e-4
SEED              = 42

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED); np.random.seed(SEED)

# ---------------------- 0) 주소/그래프 로드 -----------------------
if not os.path.exists(NODE_FEAT_CSV):
    raise FileNotFoundError(f"{NODE_FEAT_CSV} 없음")
if not os.path.exists(GRAPH_PKL):
    raise FileNotFoundError(f"{GRAPH_PKL} 없음")

node_df = pd.read_csv(NODE_FEAT_CSV)
addresses_cur = node_df['address'].astype(str).tolist()
addr2idx = {a: i for i,a in enumerate(addresses_cur)}

with open(GRAPH_PKL, "rb") as f:
    import networkx as nx
    G_nx = pickle.load(f)  # MultiDiGraph

edges = []
for u, v, _k in G_nx.edges(keys=True):
    if u in addr2idx and v in addr2idx:
        edges.append([addr2idx[u], addr2idx[v]])
edge_index_cpu = torch.tensor(edges, dtype=torch.long).t().contiguous()  # [2,E] CPU
num_nodes = len(addresses_cur)
print(f"DEVICE: {DEVICE} | nodes: {num_nodes} | edges: {edge_index_cpu.size(1)}")

# (선택) 양방향 추가: 안정성↑
# edge_index_cpu = torch.cat([edge_index_cpu, edge_index_cpu.flip(0)], dim=1)

# ---------------------- 1) Node2Vec 로드/학습 ----------------------
def try_load_saved_n2v():
    if not (os.path.exists(EMB_NPY) and os.path.exists(EDGE_PT) and os.path.exists(ADDR_CSV)):
        return None
    addr_saved = pd.read_csv(ADDR_CSV)['address'].astype(str).tolist()
    if len(addr_saved)!=len(addresses_cur) or any(a!=b for a,b in zip(addr_saved, addresses_cur)):
        print("⚠️ 저장된 addresses와 현재가 불일치 → N2V 재학습")
        return None
    x_np = np.load(EMB_NPY)
    if x_np.shape!=(len(addresses_cur), 64):
        print("⚠️ 저장된 임베딩 크기 불일치 → N2V 재학습")
        return None
    edge_saved = torch.load(EDGE_PT, map_location="cpu")
    if edge_saved.dtype!=torch.long or edge_saved.dim()!=2 or edge_saved.size(0)!=2:
        print("⚠️ 저장된 edge_index 형식 이상 → N2V 재학습")
        return None
    print("✅ 저장된 Node2Vec 임베딩/엣지/주소 사용")
    return torch.tensor(x_np, dtype=torch.float32), edge_saved, addr_saved

x_n2v_cpu = None
if not FORCE_RETRAIN_N2V:
    loaded = try_load_saved_n2v()
    if loaded:
        x_n2v_cpu, edge_index_cpu, addresses = loaded

if x_n2v_cpu is None:
    print("🛠️ Node2Vec를 새로 학습합니다...")
    data_cpu = Data(edge_index=edge_index_cpu, num_nodes=num_nodes)
    n2v = Node2Vec(
        data_cpu.edge_index, embedding_dim=64,
        walk_length=30, context_size=10, walks_per_node=200,
        p=1.0, q=1.0, num_negative_samples=1, sparse=True
    )
    n2v_loader = n2v.loader(batch_size=128, shuffle=True, num_workers=0, persistent_workers=False)
    n2v_opt = torch.optim.SparseAdam(list(n2v.parameters()), lr=0.01)
    def train_n2v(epochs=5):
        n2v.train()
        for ep in range(1, epochs+1):
            tot=0.0
            for pos_rw, neg_rw in n2v_loader:
                n2v_opt.zero_grad()
                loss = n2v.loss(pos_rw, neg_rw)
                loss.backward(); n2v_opt.step()
                tot += loss.item()
            print(f"[Node2Vec] epoch {ep:03d} | loss {tot/len(n2v_loader):.4f}")
    train_n2v(epochs=5)
    with torch.no_grad():
        x_n2v_cpu = n2v.embedding.weight.clone().detach()   # (N,64)
    # 저장
    np.save(EMB_NPY, x_n2v_cpu.numpy())
    torch.save(edge_index_cpu, EDGE_PT)
    pd.Series(addresses_cur, name="address").to_csv(ADDR_CSV, index=False)
    try: torch.save(n2v.state_dict(), N2V_PT)
    except Exception as e: print(f"(참고) Node2Vec state 저장 생략: {e}")
    addresses = addresses_cur
    print("✅ Node2Vec 저장 완료")

# ---------------- 2) 29속성과 결합 → 64차 투영(PCA) ----------------
node = pd.read_csv(NODE_FEAT_CSV).set_index("address")
feat_cols = [c for c in node.columns]
X_attr = node.reindex(addresses)[feat_cols].replace([np.inf,-np.inf], np.nan).fillna(0).astype(float).values  # (N,29)

if os.path.exists(SCALER_P) and os.path.exists(PCA64_P):
    try:
        scaler = load(SCALER_P); pca64 = load(PCA64_P)
        print("✅ 저장된 Scaler/PCA 로드")
    except Exception:
        scaler = StandardScaler().fit(X_attr)
        X_concat_fit = np.hstack([x_n2v_cpu.numpy(), scaler.transform(X_attr)])
        pca64  = PCA(n_components=64, random_state=SEED).fit(X_concat_fit)
        dump(scaler, SCALER_P); dump(pca64, PCA64_P)
else:
    scaler = StandardScaler().fit(X_attr)
    X_concat_fit = np.hstack([x_n2v_cpu.numpy(), scaler.transform(X_attr)])
    pca64  = PCA(n_components=64, random_state=SEED).fit(X_concat_fit)
    dump(scaler, SCALER_P); dump(pca64, PCA64_P)

X_concat = np.hstack([x_n2v_cpu.numpy(), scaler.transform(X_attr)])   # (N,93)
X64 = pca64.transform(X_concat).astype(np.float32)                     # (N,64)
np.save(COMB_EMB_NPY, X64)
print("✅ 결합 임베딩 저장:", COMB_EMB_NPY)

x_all = torch.tensor(X64, dtype=torch.float32, device=DEVICE)  # 전체 노드 특성(64)
data_full = Data(edge_index=edge_index_cpu, num_nodes=num_nodes)  # 특징은 외부 x_all 사용

# ----------------------- 3) 미니배치 로더 --------------------------
train_loader = NeighborLoader(
    data_full,
    input_nodes=torch.arange(num_nodes),      # 전체 노드 대상
    num_neighbors=FANOUTS,                    # 레이어별 이웃 수
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    persistent_workers=PERSISTENT
)
eval_loader = NeighborLoader(
    data_full,
    input_nodes=torch.arange(num_nodes),
    num_neighbors=FANOUTS,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    persistent_workers=False
)

# ----------------------- 4) 모델 정의 ------------------------------
class GAT_GAE(nn.Module):
    def __init__(self, in_dim=64, dropout=0.3):
        super().__init__()
        self.do = dropout
        self.g1 = GATConv(in_dim, 32, heads=8, concat=True, dropout=dropout)  # 64 -> 256
        self.g2 = GATConv(256, 32, heads=1, concat=True, dropout=dropout)     # 256 -> 32 (z)
        self.g3 = GATConv(32, 64, heads=1, concat=True, dropout=dropout)      # 32 -> 64 (x_hat)
    def forward(self, x, ei):
        x = F.dropout(x, p=self.do, training=self.training); x = F.elu(self.g1(x, ei))
        x = F.dropout(x, p=self.do, training=self.training); z = F.elu(self.g2(x, ei))
        x_hat = F.dropout(z, p=self.do, training=self.training); x_hat = self.g3(x_hat, ei)
        return x_hat, z

class SAGE_GAE(nn.Module):
    def __init__(self, in_dim=64, dropout=0.3, aggr="mean"):
        super().__init__()
        self.do = dropout
        self.s1 = SAGEConv(in_dim, 256, aggr=aggr)
        self.s2 = SAGEConv(256, 32, aggr=aggr)
        self.s3 = SAGEConv(32, 64, aggr=aggr)
    def forward(self, x, ei):
        x = F.dropout(x, p=self.do, training=self.training); x = F.elu(self.s1(x, ei))
        z = F.dropout(x, p=self.do, training=self.training); z = F.elu(self.s2(z, ei))
        x_hat = F.dropout(z, p=self.do, training=self.training); x_hat = self.s3(x_hat, ei)
        return x_hat, z

def local_link_bce(z_local, edge_index_local, batch_size):
    """
    z_local: (N_batch_all, d)   # 미니배치 서브그래프의 모든 노드 임베딩
    edge_index_local: [2, E_local]  # 로컬 인덱스(0..N_batch_all-1)
    batch_size: seed 노드 개수 (0..batch_size-1 가 seed)
    -> seed 내부(edge 양 끝이 seed에 모두 속하는) 양성 간선만 사용
    """
    src, dst = edge_index_local
    seed_mask = (src < batch_size) & (dst < batch_size)
    pos = edge_index_local[:, seed_mask]
    if pos.numel() == 0:
        # seed 내부 간선이 없으면 seed 범위에서 임의 음성만 사용 (양성 0 → BCE만으론 학습 의미 적음)
        # 안전하게 아주 작은 값 반환
        return torch.tensor(0.0, device=z_local.device)
    # 로컬 seed 서브그래프 기준으로 negative_sampling
    num_neg = pos.size(1)  # pos 수만큼
    neg = negative_sampling(pos, num_nodes=batch_size, num_neg_samples=num_neg, method='sparse')
    # 디코더(점곱)
    z_seed = z_local[:batch_size]
    def dot_decode(z, e): return (z[e[0]] * z[e[1]]).sum(dim=1)
    logits = torch.cat([dot_decode(z_seed, pos), dot_decode(z_seed, neg)], dim=0)
    labels = torch.cat([torch.ones(pos.size(1), device=logits.device),
                        torch.zeros(neg.size(1), device=logits.device)], dim=0)
    return F.binary_cross_entropy_with_logits(logits, labels)

# ----------------------- 5) 미니배치 학습 루틴 ----------------------
def train_minibatch(model, epochs, name):
    model = model.to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

    for ep in range(1, epochs+1):
        model.train()
        loss_running = mse_running = bce_running = 0.0
        steps = 0

        for batch in train_loader:
            # batch: 로컬 서브그래프
            # batch.n_id: 글로벌 노드 인덱스 (길이 = N_batch_all)
            # batch.batch_size: seed 노드 개수
            n_id = batch.n_id.to(DEVICE)               # 글로벌 → 장치
            ei   = batch.edge_index.to(DEVICE)         # 로컬 엣지
            x_mb = x_all[n_id]                         # 로컬 노드 특징(순서 = 로컬 인덱스)

            opt.zero_grad()
            x_hat, z = model(x_mb, ei)

            # MSE: seed 노드만
            bs = batch.batch_size
            mse = F.mse_loss(x_hat[:bs], x_mb[:bs])

            # 링크 BCE: seed 내부 간선만
            bce = local_link_bce(z, ei, bs)

            loss = mse + LAMBDA_BCE * bce
            loss.backward(); opt.step()

            loss_running += float(loss.item())
            mse_running  += float(mse.item())
            bce_running  += float(bce.item())
            steps += 1

        if ep == 1 or ep % 50 == 0:
            print(f"[{name}] {ep:04d} | loss {loss_running/steps:.6f} | mse {mse_running/steps:.6f} | bce {bce_running/steps:.6f}")

    # ------------- 평가(미니배치로 전체 seed 재구성 붙이기) -------------
    model.eval()
    recon_err = np.zeros(num_nodes, dtype=np.float32)

    with torch.no_grad():
        for batch in eval_loader:
            n_id = batch.n_id.to(DEVICE)
            ei   = batch.edge_index.to(DEVICE)
            x_mb = x_all[n_id]

            x_hat, _z = model(x_mb, ei)
            bs = batch.batch_size
            # seed 대응 글로벌 인덱스
            seeds_global = n_id[:bs]
            # per-node MSE
            err = torch.mean((x_hat[:bs] - x_mb[:bs])**2, dim=1).detach().cpu().numpy()
            recon_err[seeds_global.cpu().numpy()] = err

    mu, sd = recon_err.mean(), recon_err.std(ddof=1) if recon_err.size > 1 else 1e-8
    zscore = (recon_err - mu) / (sd if sd > 0 else 1e-8)
    cut = np.percentile(zscore, 95.0)
    anom = (zscore > cut).astype(int)

    out = pd.DataFrame({"address": addresses, "recon_mse": recon_err, "z_score": zscore, "is_anomaly_top5pct": anom})
    csv_name = f"{name.lower()}_mb_anomalies.csv"
    pt_name  = f"{name.lower()}_mb_model.pt"
    out.sort_values("z_score", ascending=False).to_csv(csv_name, index=False, encoding="utf-8")
    torch.save(model.state_dict(), pt_name)
    print(f"✅ {name} (mini-batch): 저장 완료 → {csv_name}, {pt_name}")
    return out

# -------------------- 6) 두 모델 연속 실행 ------------------------
gat_out  = train_minibatch(GAT_GAE(in_dim=64, dropout=DROPOUT),  EPOCHS, "GAT_GAE")
sage_out = train_minibatch(SAGE_GAE(in_dim=64, dropout=DROPOUT), EPOCHS, "SAGE_GAE")

print("✅ 완료: combined_embeddings_64.npy / scaler.joblib / pca64.joblib 저장")
print("ℹ️ GAT_GAE → gat_gae_mb_anomalies.csv, SAGE_GAE → sage_gae_mb_anomalies.csv")

DEVICE: cpu | nodes: 7958 | edges: 25601
✅ 저장된 Node2Vec 임베딩/엣지/주소 사용
✅ 저장된 Scaler/PCA 로드
✅ 결합 임베딩 저장: combined_embeddings_64.npy
[GAT_GAE] 0001 | loss 242.304894 | mse 71.805924 | bce 340.997938


C:\Users\user\AppData\Local\Temp\ipykernel_30980\3282253683.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  edge_saved = torch.load(EDGE_PT, map_location="cpu")


[GAT_GAE] 0050 | loss 3.195619 | mse 1.206307 | bce 3.978624
[GAT_GAE] 0100 | loss 1.972099 | mse 0.952752 | bce 2.038694
[GAT_GAE] 0150 | loss 1.109245 | mse 0.702397 | bce 0.813695
[GAT_GAE] 0200 | loss 1.006012 | mse 0.599225 | bce 0.813574
[GAT_GAE] 0250 | loss 0.981288 | mse 0.597751 | bce 0.767074
[GAT_GAE] 0300 | loss 0.906257 | mse 0.566926 | bce 0.678661
[GAT_GAE] 0350 | loss 1.046467 | mse 0.609092 | bce 0.874750
[GAT_GAE] 0400 | loss 0.929497 | mse 0.557273 | bce 0.744447
[GAT_GAE] 0450 | loss 0.822681 | mse 0.518042 | bce 0.609278
[GAT_GAE] 0500 | loss 0.849261 | mse 0.536495 | bce 0.625531
[GAT_GAE] 0550 | loss 0.861889 | mse 0.521575 | bce 0.680628
[GAT_GAE] 0600 | loss 0.815565 | mse 0.507944 | bce 0.615243
[GAT_GAE] 0650 | loss 0.958153 | mse 0.503514 | bce 0.909279
[GAT_GAE] 0700 | loss 0.803176 | mse 0.501523 | bce 0.603306
[GAT_GAE] 0750 | loss 0.752475 | mse 0.491933 | bce 0.521083
[GAT_GAE] 0800 | loss 0.788474 | mse 0.473126 | bce 0.630697
[GAT_GAE] 0850 | loss 0.

In [6]:
gat_ae_an = pd.read_csv("gat_ae_anomalies.csv")
sage_ae_an = pd.read_csv("sage_ae_anomalies.csv")

gat_gae_an = pd.read_csv("gat_gae_anomalies.csv")
sage_gae_an = pd.read_csv("sage_gae_anomalies.csv")

gat_gae_mb_an = pd.read_csv("gat_gae_mb_anomalies.csv")
sage_gae_mb_an = pd.read_csv("sage_gae_mb_anomalies.csv")

In [10]:
gat_ae_an[gat_ae_an['is_anomaly_top5pct'] == 1].head(10)

,address,recon_mse,z_score,is_anomaly_top5pct
0,0x1938a448d105d26c40a52a1bfe99b8ca7a745ad0,1.074462,16.639975,1
1,0x9a9eb7e103230d3baf2bd2ddc7eae69dbb3f77b8,0.979735,15.079760,1
2,0x167a9333bf582556f35bd4d16a7e80e191aa6476,0.973421,14.975760,1
3,0xf37b1a35647e4efc1afec5bb870e0bcbf1ac2ffc,0.911353,13.953463,1
4,0xf204a7552bb25302a70f8695c7d5edbc8e32cb85,0.897239,13.720983,1
5,0xb5f756611eddfbd63f4e8d28f2a62a401431c35a,0.871787,13.301768,1
6,0x11784e0732270b41dd7aba1baa266f076b78f085,0.844250,12.848221,1
7,0xaf0ae50cd011e741cdb90f624b5ff0f06fd6ef58,0.839779,12.774575,1
8,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,0.829123,12.599068,1
9,0x19095a519eccd68213b6aa7a80577337d291006e,0.784288,11.860599,1


In [11]:
sage_ae_an[sage_ae_an['is_anomaly_top5pct'] == 1].head(10)

,address,recon_mse,z_score,is_anomaly_top5pct
0,0x9a9eb7e103230d3baf2bd2ddc7eae69dbb3f77b8,0.838493,14.989171,1
1,0x167a9333bf582556f35bd4d16a7e80e191aa6476,0.800249,14.254773,1
2,0x1938a448d105d26c40a52a1bfe99b8ca7a745ad0,0.768789,13.650672,1
3,0xf37b1a35647e4efc1afec5bb870e0bcbf1ac2ffc,0.764231,13.563149,1
4,0xf204a7552bb25302a70f8695c7d5edbc8e32cb85,0.760840,13.498028,1
5,0xb5f756611eddfbd63f4e8d28f2a62a401431c35a,0.728500,12.877007,1
6,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,0.727814,12.863837,1
7,0x11784e0732270b41dd7aba1baa266f076b78f085,0.703027,12.387865,1
8,0xaf0ae50cd011e741cdb90f624b5ff0f06fd6ef58,0.683921,12.020988,1
9,0x19095a519eccd68213b6aa7a80577337d291006e,0.681271,11.970102,1


In [12]:
gat_gae_an[gat_gae_an['is_anomaly_top5pct'] == 1].head(10)

,address,recon_mse,z_score,is_anomaly_top5pct
0,0x167a9333bf582556f35bd4d16a7e80e191aa6476,982.400940,73.656010,1
1,0x0ce15800ebc76a1034d320ad2abcd0e77ba52ece,295.798220,22.151089,1
2,0x1f69b6147203344049ea381f5dd2008714caedda,280.451140,20.999840,1
3,0xd28493e737fbcc957f3716143ed6e40f40357b51,275.396060,20.620638,1
4,0xd4b394c60bb55f80df30dac87b6f92be34739332,254.439870,19.048626,1
5,0x0000000000000000000000000000000000000000,232.739840,17.420818,1
6,0x1938a448d105d26c40a52a1bfe99b8ca7a745ad0,205.130690,15.349741,1
7,0xa5025faba6e70b84f74e9b1113e5f7f4e7f4859f,123.233376,9.206282,1
8,0xfea2c6d96f0bd4cd4d638bcefa7968146c74df37,110.310020,8.236848,1
9,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,78.658590,5.862543,1


In [13]:
sage_gae_an[sage_gae_an['is_anomaly_top5pct'] == 1].head(10)

,address,recon_mse,z_score,is_anomaly_top5pct
0,0x167a9333bf582556f35bd4d16a7e80e191aa6476,980.456400,73.610580,1
1,0x0ce15800ebc76a1034d320ad2abcd0e77ba52ece,293.959170,22.043434,1
2,0x1f69b6147203344049ea381f5dd2008714caedda,279.166100,20.932234,1
3,0xd28493e737fbcc957f3716143ed6e40f40357b51,273.299320,20.491543,1
4,0xd4b394c60bb55f80df30dac87b6f92be34739332,250.081900,18.747536,1
5,0x0000000000000000000000000000000000000000,246.659940,18.490492,1
6,0x1938a448d105d26c40a52a1bfe99b8ca7a745ad0,204.375640,15.314251,1
7,0xa5025faba6e70b84f74e9b1113e5f7f4e7f4859f,120.477280,9.012116,1
8,0xfea2c6d96f0bd4cd4d638bcefa7968146c74df37,110.667700,8.275256,1
9,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,77.889534,5.813081,1


In [14]:
gat_gae_mb_an[gat_gae_mb_an['is_anomaly_top5pct'] == 1].head(10)

,address,recon_mse,z_score,is_anomaly_top5pct
0,0x167a9333bf582556f35bd4d16a7e80e191aa6476,982.01010,74.976320,1
1,0x0ce15800ebc76a1034d320ad2abcd0e77ba52ece,294.19693,22.437508,1
2,0x1f69b6147203344049ea381f5dd2008714caedda,279.11370,21.285372,1
3,0xd4b394c60bb55f80df30dac87b6f92be34739332,254.99422,19.442997,1
4,0xd28493e737fbcc957f3716143ed6e40f40357b51,213.19151,16.249886,1
5,0x1938a448d105d26c40a52a1bfe99b8ca7a745ad0,204.81764,15.610245,1
6,0x0000000000000000000000000000000000000000,191.34544,14.581166,1
7,0xa5025faba6e70b84f74e9b1113e5f7f4e7f4859f,121.95533,9.280783,1
8,0xfea2c6d96f0bd4cd4d638bcefa7968146c74df37,111.11345,8.452622,1
9,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,78.07902,5.929277,1


In [15]:
sage_gae_mb_an[sage_gae_mb_an['is_anomaly_top5pct'] == 1].head(10)

,address,recon_mse,z_score,is_anomaly_top5pct
0,0x167a9333bf582556f35bd4d16a7e80e191aa6476,970.829200,79.920140,1
1,0x1f69b6147203344049ea381f5dd2008714caedda,278.117030,22.873000,1
2,0x0ce15800ebc76a1034d320ad2abcd0e77ba52ece,271.768980,22.350216,1
3,0x1938a448d105d26c40a52a1bfe99b8ca7a745ad0,183.511540,15.081924,1
4,0xd28493e737fbcc957f3716143ed6e40f40357b51,119.247780,9.789589,1
5,0xfea2c6d96f0bd4cd4d638bcefa7968146c74df37,91.081800,7.470028,1
6,0x0000000000000000000000000000000000000000,81.365660,6.669871,1
7,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,74.387160,6.095169,1
8,0xd4b394c60bb55f80df30dac87b6f92be34739332,64.854290,5.310105,1
9,0xf204a7552bb25302a70f8695c7d5edbc8e32cb85,58.322456,4.772187,1


In [ ]:
gat_ae_anomalies, sage_ae_anomalies, gat_gae_anomalies, sage_gae_anomalies, gat_gae_mb_anomalies, sage_gae_mb_anomalies

In [28]:
import pandas as pd
import numpy as np
from pathlib import Path

# ============================================
# 파일명만 바꿔서 실행하세요 (예시)
file_a = "gat_ae_anomalies"   # 예: "gat_ae_an", "sage_gae_an", ...
file_b = "sage_ae_anomalies"
# ============================================

def ensure_csv(path_like: str) -> str:
    return path_like if str(path_like).lower().endswith(".csv") else str(path_like) + ".csv"

def load_anomaly_addresses(csv_path: str) -> pd.Series:
    df = pd.read_csv(csv_path)
    # 컬럼 체크
    required = {"address", "is_anomaly_top5pct"}
    if not required.issubset(df.columns):
        missing = required - set(df.columns)
        raise ValueError(f"{csv_path}: 필요한 컬럼이 없습니다 → {missing}")
    # 필터링: is_anomaly_top5pct == 1
    s = df.loc[(df["is_anomaly_top5pct"].astype(float) == 1), "address"]
    # 주소 정규화(소문자/공백 제거)
    s = s.astype(str).str.strip().str.lower()
    return s.dropna()

def compare_two(csv_a: str, csv_b: str):
    csv_a = ensure_csv(csv_a)
    csv_b = ensure_csv(csv_b)

    a = load_anomaly_addresses(csv_a)
    b = load_anomaly_addresses(csv_b)

    A = set(a.unique())
    B = set(b.unique())
    inter = A & B
    union = A | B

    # 지표들
    nA, nB, nI, nU = len(A), len(B), len(inter), len(union)
    jaccard = nI / nU if nU else 0.0
    overlap_coeff = nI / min(nA, nB) if min(nA, nB) else 0.0
    prec_A_to_B = nI / nA if nA else 0.0
    rec_A_to_B  = nI / nB if nB else 0.0
    f1 = (2*prec_A_to_B*rec_A_to_B)/(prec_A_to_B+rec_A_to_B) if (prec_A_to_B+rec_A_to_B)>0 else 0.0

    print(f"=== 비교 결과: {Path(csv_a).name}  vs  {Path(csv_b).name} ===")
    print(f"A(=왼쪽) 이상치 수: {nA}")
    print(f"B(=오른쪽) 이상치 수: {nB}")
    print(f"교집합: {nI} / 합집합: {nU}")
    print(f"Jaccard: {jaccard:.4f}")
    print(f"Overlap Coefficient: {overlap_coeff:.4f}  (|A∩B| / min(|A|,|B|))")
    print(f"Precision(A→B): {prec_A_to_B:.4f}   Recall(A→B): {rec_A_to_B:.4f}   F1: {f1:.4f}")

    # 결과를 파일로도 저장(원하면 주석 해제)
    stemA, stemB = Path(csv_a).stem, Path(csv_b).stem
    pd.Series(sorted(inter), name="address").to_csv(f"common_{stemA}__{stemB}.csv", index=False)
    pd.Series(sorted(A - B), name="address").to_csv(f"only_{stemA}.csv", index=False)
    pd.Series(sorted(B - A), name="address").to_csv(f"only_{stemB}.csv", index=False)
    print(f"파일 저장 → common_{stemA}__{stemB}.csv, only_{stemA}.csv, only_{stemB}.csv")

# 실행
compare_two(file_a, file_b)

=== 비교 결과: gat_ae_anomalies.csv  vs  sage_ae_anomalies.csv ===
A(=왼쪽) 이상치 수: 398
B(=오른쪽) 이상치 수: 398
교집합: 385 / 합집합: 411
Jaccard: 0.9367
Overlap Coefficient: 0.9673  (|A∩B| / min(|A|,|B|))
Precision(A→B): 0.9673   Recall(A→B): 0.9673   F1: 0.9673
파일 저장 → common_gat_ae_anomalies__sage_ae_anomalies.csv, only_gat_ae_anomalies.csv, only_sage_ae_anomalies.csv
